In [2]:
# Import Required Libraries
import tensorflow as tf  
# TensorFlow → used to load and run the trained ANN model

from tensorflow.keras.models import load_model  
# load_model → loads the saved .h5 ANN model

import pickle  
# pickle → used to load saved encoders and scaler

import pandas as pd  
# pandas → used to create and manipulate DataFrames

import numpy as np  
# numpy → numerical operations (arrays, matrices)

c:\Users\subha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
# Load the Trained ANN Model
model = load_model('model.h5')  
# Load the trained ANN model saved during training
# This model already knows the learned weights

In [4]:
# Load Saved Encoders and Scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)  
# Load OneHotEncoder for Geography
# Ensures same encoding as training phase

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)  
# Load LabelEncoder for Gender
# Prevents mismatch in Male/Female encoding

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)  
# Load StandardScaler
# Scaling must be identical to training

In [6]:
# Example Input Data (New Customer)
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}  
# This dictionary represents a new customer's details
# Model will predict churn for this customer
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [7]:
# One-Hot Encode Geography
geo_encoded = label_encoder_geo.transform(
    [[input_data['Geography']]]
).toarray()  
# Convert Geography value into one-hot encoded numeric form
print(geo_encoded)

[[1. 0. 0.]]


c:\Users\subha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [8]:
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)  
# Convert encoded array into DataFrame with proper column names
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
# Convert Input Dictionary to DataFrame
input_df = pd.DataFrame([input_data])  
# Convert input dictionary into pandas DataFrame
# Required for further preprocessing
print(input_df)

   CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  


In [10]:
# Encode Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])  
# Convert Gender (Male/Female) → numeric (0/1)
# Must match training encoding
print(input_df)

   CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France       1   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  


In [11]:
# Merge Geography Encoded Columns
input_df = pd.concat(
    [input_df.drop("Geography", axis=1), geo_encoded_df],
    axis=1
)  
# Remove original Geography column
# Add one-hot encoded Geography columns
print(input_df)

   CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0  


In [13]:
# Scale the Input Data
input_scaled = scaler.transform(input_df)  
# Scale input features using same scaler as training
# ANN works best with scaled numerical data
print(input_scaled)

[[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


In [14]:
# Predict Customer Churn
prediction = model.predict(input_scaled)  
# Model outputs probability between 0 and 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


In [17]:
prediction_proba = prediction[0][0]  
# Extract probability value from model output
print(prediction_proba)

0.021899328


In [18]:
# Final Decision Based on Threshold
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')  
# Threshold = 0.5
# > 0.5 → Churn
# ≤ 0.5 → No Churn

The customer is not likely to churn.
